##### Import Package

In [2]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn.cross_validation import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from konlpy.tag import Twitter

import pandas as pd

##### Get Article

In [3]:
df_list = []
for day in range(1,31):
    day = "0" + str(day) if 10 > day else str(day)
    date = "2016-06-" + day
    df = pd.read_csv("./news/" + date + ".csv").fillna("-")
    df_list.append(df)

for day in range(1,30):
    day = "0" + str(day) if 10 > day else str(day)
    date = "2016-05-" + day
    df = pd.read_csv("./news/" + date + ".csv").fillna("-")
    df_list.append(df)

article_df = pd.concat(df_list).reset_index(drop=True)

len(article_df), article_df.columns

(104960,
 Index(['newsid', 'oid', 'newspaper', 'title', 'link', 'comment', 'likeit',
        'content', 'date', 'category'],
       dtype='object'))

##### Split Tratin, Test

In [4]:
X_train, X_test, y_train, y_test = train_test_split(article_df.content, article_df.category, test_size=0.01, random_state=1)
len(X_train), len(X_test), len(y_train), len(y_test)

(103910, 1050, 103910, 1050)

##### Set Stop Words

In [5]:
stop_words_list = ["동아일보", "경향신문"]

##### Set Morpheme

In [6]:
pos_tagger = Twitter()

def tokenize_pos(doc):
    return ['/'.join(t) for t in pos_tagger.pos(doc, norm=True, stem=True)]

##### Set Model

In [7]:
clf = Pipeline([
#             ('vect', TfidfVectorizer(tokenizer=tokenize_pos, stop_words=stop_words, ngram_range=(1,2))),
            ('vect', TfidfVectorizer(stop_words=stop_words_list, ngram_range=(1,2))),
            ('clf', MultinomialNB(alpha=0.01)),
        ])

In [8]:
%time model = clf.fit(X_train[:10000],y_train[:10000]) 

CPU times: user 29.9 s, sys: 1.16 s, total: 31 s
Wall time: 31.1 s


##### Confusion Matrix, Classfication Report
- category - 0:정치, 1:경제, 2:사회, 3.생활/문화, 4.세계, 5.IT/과학

In [9]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

%time y_pred = model.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

CPU times: user 1.42 s, sys: 161 ms, total: 1.58 s
Wall time: 1.6 s
[[242   7   7   0   3   0]
 [  6 145  12   3   2   1]
 [ 12  12 275   4   1   0]
 [  1   9  20  83   3   0]
 [  5   8  10   2 124   0]
 [  2  11   4   3   6  27]]
             precision    recall  f1-score   support

          0       0.90      0.93      0.92       259
          1       0.76      0.86      0.80       169
          2       0.84      0.90      0.87       304
          3       0.87      0.72      0.79       116
          4       0.89      0.83      0.86       149
          5       0.96      0.51      0.67        53

avg / total       0.86      0.85      0.85      1050



In [10]:
X_test = X_test.reset_index(drop=True)
X_test

0       유일호 부총리 겸 기획재정부 장관은 14일 “충분한 재정 보강책을 고민 중이며, 조...
1       정의화 국회의장이 17일 오전(현지시간) 일본 중의원 제1의원회관에서 개최된 '한일...
2       【서울=뉴시스】이수지 기자 = 미국 재무부가 1일(현지시간) 북한을 '주요 자금세탁...
3       옐로스톤 국립공원(와이오밍).그랜드캐니언 국립공원(애리조나).디날리 국립공원(알래스...
4       [뉴스닥] 장하나 "상임위 배치 논란 제기 추혜선 상한가" vs 이준석 "청년 정치...
5       국회 정무위원회 업무보고 첫날인 29일, 국회와 금융위원회 간의 서별관회의 자료 제...
6       국제범죄 대응 주제 발표.. 100여개 회원국 인사 참여강신명 경찰청장이 오는 3일...
7       [머니투데이 김지민 기자] [삼성SDS "물류사업 분할·남은 사업 경쟁력 확보 방안...
8       주요 증권사들의 1분기 실적이 대형 증권사를 중심으로 저조한 가운데 발 빠르게 자신...
9       1일 새누리당 정진석 당선자(충남 공주·부여·청양·4선)이 신임 원내대표 경선 출마...
10      조지아-나토군 연합훈련에 참가한 미군 에이브람스 탱크 [EPA=연합뉴스]푸틴 "미국...
11      [머니투데이 김만배 기자, 이태성 기자, 양성희 기자, 한정수 기자, 김종훈 기자,...
12      대기업 수가 절반 가까이 줄면 세수에는 어떤 영향이 있을까. 신영선 공정거래위원회 ...
13      ▶ [핫클릭] 근로정신대 양금덕 할머니, 송혜교씨에 감사 편지▶ [핫클릭] '중국 ...
14      【서울=뉴시스】박주성 기자 = 한민구 국방부 장관이 4일(현지시간) 싱가포르 샹그릴...
15            17일 오전 12시 20분께 부산 해운대구 우동의 한 유명 고깃집 1층에...
16      [머니투데이 남형도 기자] [서울연구원, 외국인 관광객 300명 심층인터뷰 해 '불...
17      - 여객전용

In [11]:
# 0:정치, 1:경제, 2:사회, 3.생활/문화, 4.세계, 5.IT/과학
classification_dict = {
    0:"정치",
    1:"경제",
    2:"사회",
    3:"생활/문화",
    4:"세계",
    5:"IT/과학",
}
result = model.predict([ X_test[551], X_test[556], X_test[559] ])
for category in result:
    print(classification_dict[category])

경제
세계
세계


##### save model

In [12]:
import pickle

pickle.dump(model, open("classification_model.plk", "wb"))

In [13]:
load_model = pickle.load(open("classification_model.plk", "rb"))

In [17]:
test_str1 = "카카오 주식 정보 제공을 위한 서비스 런칭"
test_str2 = "요즘에 환율과 주가가 너무 오르락 내리락"
print( classification_dict[ load_model.predict([test_str1])[0] ] )
print( classification_dict[ load_model.predict([test_str2])[0] ] )

IT/과학
경제
